# Wheel Strategy Options Filtering

This notebook implements comprehensive filtering for the Wheel Strategy:
- DTE (Days to Expiration) filtering
- Delta filtering
- ROC (Return on Capital) filtering
- Liquidity filtering
- Combined filter pipeline

In [ ]:
import pandas as pd
import numpy as np
from py_vollib.black_scholes.greeks.analytical import delta
import databento as db
from datetime import timedelta

In [ ]:
# Backtest configuration for Wheel Strategy
BACKTEST_CONFIG = {
    # DTE (Days to Expiration) - in trading days
    'min_dte': 30,
    'max_dte': 45,
    
    # Delta range (absolute values for puts)
    'min_delta': 0.20,  # -0.20 delta
    'max_delta': 0.40,  # -0.40 delta
    
    # ROC (Return on Capital) targets
    'min_roc': 2.0,  # 2% minimum
    'max_roc': 10.0,  # 10% maximum (avoid excessive risk)
    
    # Liquidity requirements
    'min_open_interest': 100,
    'min_volume': 10,
    'max_spread_pct': 10.0,  # Max 10% bid-ask spread
    
    # Pricing
    'use_mid_price': False,  # False = use bid (conservative), True = use mid
    
    # Risk parameters
    'risk_free_rate': 0.05,  # 5% for delta calculation
    'default_iv': 0.30,  # 30% default implied volatility
}

print("Backtest configuration loaded:")
for key, value in BACKTEST_CONFIG.items():
    print(f"  {key}: {value}")

## Helper Functions

In [ ]:
def calculate_trading_days(event_date, exp_date, holidays=None):
    """
    Calculate trading days between two dates (excludes weekends and holidays)
    
    Parameters:
    - event_date: Starting date
    - exp_date: Expiration date
    - holidays: Optional array of holiday dates
    
    Returns:
    - Number of trading days
    """
    return np.busday_count(event_date, exp_date, holidays=holidays)

In [ ]:
def calculate_delta_bs(spot_price, strike, dte_days, risk_free_rate, volatility, option_type='p'):
    """
    Calculate Black-Scholes delta for an option
    
    Parameters:
    - spot_price: Current underlying price
    - strike: Strike price
    - dte_days: Days to expiration
    - risk_free_rate: Risk-free rate (annualized)
    - volatility: Implied volatility (annualized)
    - option_type: 'p' for put, 'c' for call
    
    Returns:
    - Delta value
    """
    try:
        t = dte_days / 365.0  # Convert to years
        return delta(option_type, spot_price, strike, t, risk_free_rate, volatility)
    except Exception as e:
        return np.nan

In [ ]:
def calculate_roc(premium, strike):
    """
    Calculate Return on Capital for a cash-secured put
    
    ROC = (Premium / (Strike * 100)) * 100
    
    Parameters:
    - premium: Option premium (per share)
    - strike: Strike price
    
    Returns:
    - ROC as percentage
    """
    capital_required = strike * 100  # Strike * 100 shares
    return (premium / capital_required) * 100

In [ ]:
def calculate_spread_pct(bid, ask):
    """
    Calculate bid-ask spread as percentage of mid price
    
    Parameters:
    - bid: Bid price
    - ask: Ask price
    
    Returns:
    - Spread percentage
    """
    if bid <= 0 or ask <= 0:
        return np.nan
    mid = (bid + ask) / 2
    if mid == 0:
        return np.nan
    return ((ask - bid) / mid) * 100

## Main Filtering Function

In [ ]:
def filter_wheel_options(df, underlying_price, config=BACKTEST_CONFIG, verbose=True):
    """
    Comprehensive filtering for Wheel Strategy options
    
    Parameters:
    - df: Options dataframe with columns: strike, dte, bid, ask, volume, open_interest, call_put
    - underlying_price: Current price of underlying stock
    - config: Dictionary with filter parameters (uses BACKTEST_CONFIG by default)
    - verbose: Print filtering statistics
    
    Returns:
    - Filtered dataframe with added columns: premium, delta, abs_delta, roc_pct, spread_pct, 
      capital_required, moneyness
    """
    result = df.copy()
    initial_count = len(result)
    
    if verbose:
        print(f"Starting with {initial_count} options\n")
    
    # 1. Filter by option type (puts only for wheel strategy)
    result = result[result['call_put'] == 'P']
    if verbose:
        print(f"After put filter: {len(result)} ({len(result)/initial_count*100:.1f}%)")
    
    # 2. Filter by DTE
    result = result[
        (result['dte'] >= config['min_dte']) & 
        (result['dte'] <= config['max_dte'])
    ]
    if verbose:
        print(f"After DTE filter ({config['min_dte']}-{config['max_dte']}): {len(result)} ({len(result)/initial_count*100:.1f}%)")
    
    # 3. Calculate premium (bid or mid)
    if config['use_mid_price']:
        result['premium'] = (result['bid'] + result['ask']) / 2
    else:
        result['premium'] = result['bid']  # Conservative
    
    # 4. Calculate ROC
    result['capital_required'] = result['strike'] * 100
    result['roc_pct'] = result.apply(
        lambda row: calculate_roc(row['premium'], row['strike']), 
        axis=1
    )
    
    # Filter by ROC
    result = result[
        (result['roc_pct'] >= config['min_roc']) & 
        (result['roc_pct'] <= config['max_roc'])
    ]
    if verbose:
        print(f"After ROC filter ({config['min_roc']}-{config['max_roc']}%): {len(result)} ({len(result)/initial_count*100:.1f}%)")
    
    # 5. Calculate Delta
    result['delta'] = result.apply(
        lambda row: calculate_delta_bs(
            underlying_price, 
            row['strike'], 
            row['dte'], 
            config['risk_free_rate'], 
            config['default_iv'],
            'p'
        ),
        axis=1
    )
    result['abs_delta'] = result['delta'].abs()
    
    # Filter by delta (using absolute value)
    result = result[
        (result['abs_delta'] >= config['min_delta']) & 
        (result['abs_delta'] <= config['max_delta'])
    ]
    if verbose:
        print(f"After delta filter ({config['min_delta']}-{config['max_delta']}): {len(result)} ({len(result)/initial_count*100:.1f}%)")
    
    # 6. Calculate spread percentage
    result['spread_pct'] = result.apply(
        lambda row: calculate_spread_pct(row['bid'], row['ask']),
        axis=1
    )
    
    # 7. Filter by liquidity
    result = result[
        (result['open_interest'] >= config['min_open_interest']) & 
        (result['volume'] >= config['min_volume']) &
        (result['spread_pct'] <= config['max_spread_pct'])
    ]
    if verbose:
        print(f"After liquidity filter (OI>={config['min_open_interest']}, Vol>={config['min_volume']}, Spread<={config['max_spread_pct']}%): {len(result)} ({len(result)/initial_count*100:.1f}%)")
    
    # 8. Calculate moneyness
    result['moneyness'] = result['strike'] / underlying_price
    
    # 9. Sort by ROC (highest first)
    result = result.sort_values('roc_pct', ascending=False)
    
    if verbose:
        print(f"\n=== Final Result: {len(result)} options ({len(result)/initial_count*100:.1f}% of original) ===")
    
    return result

## Example Usage

In [ ]:
# Example: Load your options data
# df = your_options_dataframe
# underlying_price = 179.21  # AAPL spot price

# filtered_options = filter_wheel_options(df, underlying_price, BACKTEST_CONFIG)

# Display results
# filtered_options[[
#     'symbol', 'expiration', 'strike', 'dte', 'abs_delta', 
#     'premium', 'roc_pct', 'volume', 'open_interest', 'spread_pct'
# ]]

## Select Best Options

After filtering, select the best options based on your criteria

In [ ]:
def select_best_options(filtered_df, n=5, sort_by='roc_pct'):
    """
    Select top N options based on sorting criteria
    
    Parameters:
    - filtered_df: Filtered options dataframe
    - n: Number of options to select
    - sort_by: Column to sort by ('roc_pct', 'abs_delta', 'volume', etc.)
    
    Returns:
    - Top N options
    """
    return filtered_df.sort_values(sort_by, ascending=False).head(n)

## Summary Statistics

In [ ]:
def print_summary_stats(filtered_df):
    """
    Print summary statistics for filtered options
    """
    print("=== Summary Statistics ===")
    print(f"\nTotal options: {len(filtered_df)}")
    print(f"\nROC (Return on Capital):")
    print(filtered_df['roc_pct'].describe())
    print(f"\nDelta (absolute):")
    print(filtered_df['abs_delta'].describe())
    print(f"\nDTE (Days to Expiration):")
    print(filtered_df['dte'].describe())
    print(f"\nSpread (%):")
    print(filtered_df['spread_pct'].describe())
    print(f"\nOpen Interest:")
    print(filtered_df['open_interest'].describe())
    print(f"\nVolume:")
    print(filtered_df['volume'].describe())